In [1]:
import sys, os
import pandas as pd
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

import torch

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)


#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "regression"

# Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'RNN_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
model_method = 'GRU_rg'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerRootDir = './scaler/'

# 2. Training 

## 2-1. Data selection

In [3]:
get_ms_list = db_client.measurement_list(bucket_name)

/home/jwmoon/.local/lib/python3.8/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    import "influxdata/influxdb/schema" schema.measurements(bucket: "integration") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


In [4]:
# 현재 소스 기준으로 불필요한 데이터 influx와 mongo 지워주세요
get_ms_list

['Hs1SwineFarmWithWeatherTime_test_Clean',
 'Hs1SwineFarmWithWeatherTime_test_NoClean',
 'Hs1SwineFarmWithWeatherTime_train_Clean',
 'Hs1SwineFarmWithWeatherTime_train_NoClean',
 'energy_regression__testX_Clean',
 'energy_regression__testX_NoClean',
 'energy_regression__testy_Clean',
 'energy_regression__testy_NoClean',
 'energy_regression__trainX_Clean',
 'energy_regression__trainX_NoClean',
 'energy_regression__trainy_Clean',
 'energy_regression__trainy_NoClean',
 'energy_regression_testXClean',
 'energy_regression_testXNoClean',
 'energy_regression_testyClean',
 'energy_regression_testyNoClean',
 'energy_regression_trainXClean',
 'energy_regression_trainXNoClean',
 'energy_regression_trainyClean',
 'energy_regression_trainyNoClean',
 'energytestX_regression_NoClean',
 'energytesty_regression_NoClean',
 'energytrainX_regression_NoClean',
 'energytrainy_regression_NoClean',
 'forecasting_gunwiStrawberryWeather_test_Clean',
 'forecasting_gunwiStrawberryWeather_test_NoClean',
 'forecast

In [5]:
ms_list = ['ActionPatternData_testX_NoClean',
 'ActionPatternData_testy_NoClean',
 'ActionPatternData_trainX_NoClean',
 'ActionPatternData_trainy_NoClean']

In [6]:
# for regression data
if mode_selection == 'regression':

    datasetName = 'ActionPatternData'
    
    ms_name_X = 'ActionPatternData_trainX_NoClean'
    ms_name_y = 'ActionPatternData_trainy_NoClean'
    
    dataX = db_client.get_data(bucket_name, ms_name_X)
    datay = db_client.get_data(bucket_name, ms_name_y)
    

# for forecast data test
elif mode_selection == 'forecast':
    
    cleanMode = 'NoClean'
    datasetName = 'Hs1SwineFarmWithWeatherTime'
    
    ms_name_X = 'Hs1SwineFarmWithWeatherTime_train_NoClean'
    dataX = db_client.get_data(bucket_name, ms_name_X)

## 2-2. Training Data Preparation

In [7]:
# for regression
if mode_selection == 'regression':
    # 2 Training Data Preparation
    # 2-1
    featureListX= list(dataX.columns)
    featureListy= list(datay.columns)

    # 2-2
    cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

    # 2-2-1 cleanTrainDataParam == Clean 일 경우
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }
    # 2-3
    scalerParam='scale'
    scaleMethod='minmax'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerRootDir, datasetName, cleanTrainDataParam)
    scalerRootPath_y = os.path.join(scalerRootDir, datasetName, cleanTrainDataParam)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

# for forecast
elif mode_selection == 'forecast':
    # 2-1
    featureListX = ['Temperature', 'out_temp','sin_hour']
    target_col = 'Temperature'

    # 2-2
    cleanTrainDataParam = cleanMode

    # 2-2-1 cleanMode == Clean 일 때만 활성화
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }

    # 2-3
    scalerParam='scale'
    scaleMethod ='robust'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerRootDir, dataName_X, cleanMode)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_x, val_x = ml_clean.delete_low_quality_train_val_data(train_x, val_x, cleanMode, NaNProcessingParam)

[]


ValueError: at least one array or dtype is required

## 2-3. Training Data Transformation

In [ ]:
# for regression
if mode_selection == 'regression':
    transformParameter = {}

    from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

    trainX, trainy = transDFtoNP(train_x, train_y)
    valX, valy = transDFtoNP(val_x, val_y)

# for forecast
elif mode_selection == 'forecast':
    transformParameter = {
    'future_step': 2,
    'past_step': 24,
    'feature_col': featureListX,
    'target_col': target_col,
    'clean_param': cleanTrainDataParam
    }
    from Clust.clust.transformation.purpose.machineLearning import LSTMData

    LSTMD = LSTMData()
    trainX, trainy = LSTMD.transform_Xy_arr(train_x, transformParameter, transformParameter['clean_param'])
    valX, valy = LSTMD.transform_Xy_arr(val_x, transformParameter, transformParameter['clean_param'])

## 2-4 Set Model Parameters & Train Parameters

In [ ]:
input_size, seq_len = trainX.shape[1], trainX.shape[2]
print(input_size, seq_len)

# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_rg' or model_method == 'GRU_rg' or model_method == 'RNN_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

In [ ]:
collection_name = datasetName
document = { 'data_name': ms_name_X}
trainDataInfo = mongo_client.get_document_by_json(bucket_name, collection_name, document)

In [ ]:
modelTags =["aaaaa"]
trainDataType = "timeseries"
modelPurpose = "regression"

# # 2
# trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3. 모델을 저장할 파일 패스를 생성한다.

model_name = 'Test' + model_method
from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, ms_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

## 2-5 Training 

In [ ]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(trainX, trainy, valX, valy)
rml.train()
rml.save_best_model(modelFilePath)

## 2-6 Save MetaData 

In [ ]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
# trainParameter_encode =  encode_hash_style(str(trainParameter))
modelInfoMeta ={
    "trainDataInfo":trainDataInfo,
    "modelName":model_name,
    "featureList":featureListX,
    "target": featureListy,
    "trainDataType":trainDataType,
    "modelPurpose":modelPurpose,
    "model_method":model_method,
    "modelTags":modelTags,
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":NaNProcessingParam,
    "trainDataName":[ms_name_X, ms_name_y],
    "trainParameter": trainParameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "NaNProcessingParam":NaNProcessingParam,
    "scalerParam":scalerParam,
    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":modelFilePath
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath       
        }
    }
}

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)
========== Data Svae Success ==========
{'trainDataInfo': [{'data_name': 'ActionPatternData_trainX_NoClean', 'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'startTime': '2016-01-11', 'endTime': '2016-04-15', 'cleanParam': 'NoClean'}], 'modelName': 'TestGRU_rg', 'featureList': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed'], 'target': ['value'], 'trainDataType': 'timeseries', 'modelPurpose': 'regression', 'model_method': 'GRU_rg', 'modelTags': ['aaaaa'], 'cleanTrainDataParam': 'NoClean', 'NaNProcessingParam': {'feature_cycle': 'Day', 'feature_cycle_times': 1, 'NanInfoForCleanData': {'type': 'num', 'ConsecutiveNanLimit': 3, 'totalNaNLimit': 30000}}, 'trainDataName': ['ActionPatternData_trainX_NoClean', 'ActionPatternData_trainy_NoClean'], 'trainParameter': {'lr': 0.0001, 'weight_decay': 1e-06, 'device': 'cpu', 'n_epochs': 10, 'batch_size': 16}, 'modelParameter': {'rnn_type': 'lstm', 'input_size': 24, 'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}, 'transformParameter': {}, 'scalerParam': 'scale', 'files': {'modelFile': {'fileName': 'model.pth', 'filePath': './Models/GRU_rg/TestGRU_rg/ActionPatternData_trainX_NoClean/d531eec11664669cff1f6a3ad9639012/model.pkl'}, 'XScalerFile': {'fileName': 'scaler.pkl', 'filePath': './scaler/ActionPatternData/NoClean/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'}, 'yScalerFile': {'fileName': 'scaler.pkl', 'filePath': './scaler/ActionPatternData/NoClean/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}, '_id': ObjectId('6435107d3d262e8168f7c3d0')}